In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [2]:
userhome = os.path.expanduser('~')
repository = os.path.dirname(userhome + r'/historage-mining/historage/openjpa/')

# Define function to apply git command

In [3]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

In [4]:
def execute_shell_command(id1, id2, cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return id1, id2, out, error
    pipe.wait()

# Listing all files in repository and save it in textfile

In [5]:
listfile = []
for filename in glob.iglob(userhome + r'/historage-mining/historage/openjpa/**/*.*', recursive=True):
    listfile.append(filename)

In [6]:
with open ("file_list.txt", mode="wt", encoding="utf-8") as mytxt:
    mytxt.write('\n'.join(listfile))
print ("File file_list.txt has been created")
print ("Number of files found: %i" % (len(listfile)) + " files")

File file_list.txt has been created
Number of files found: 5185 files


# Extracting Commit ID

In [7]:
command = "git log --pretty='%H'"
com_ID = str(execute_command(command, repository)).replace("b'",'').replace("'",'').replace('\\n',' ').replace("(",'').replace(")",'').replace(",",'').split(' ')

# Applying git diff -w --diff-algorithm=histogram --stat commitID_1 commitID2 

In [8]:
diffhist = []
for i in range(0, len(com_ID)-3):
    sys.stdout.write('\rApplying diff between commits = %i' % (i+1) + ' out of %i' % (len(com_ID)-3))
    sys.stdout.flush()
    diff_cmd = "git diff -w --diff-algorithm=histogram --stat " + com_ID[i+1] + " " + com_ID[i]
    tmp = (str(execute_shell_command(com_ID[i+1], com_ID[i], diff_cmd, repository)).replace("(",'').replace("+",'').
           replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').replace(")",'').
           replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
           replace("...=> ",'...=>').replace(" Code ","Code"))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                    replace(",file",'files').replace(",insertions",'insertions').
                    replace(",insertion",'insertions').replace(",deletions,",'deletions').
                    replace(",deletion,",'deletions').replace(" ",''))
    diffhist.append(str(tmp2))
    
diffresult = []
for item in diffhist:
    if item == ', ':
        del item
    else:
        diffresult.append(item)

#diffresult

Applying diff between commits = 4860 out of 4860

In [9]:
diffres = []
for x in range(0,len(diffresult)-1):
    line = diffresult[x]
    if "changed" not in diffresult[x]:
        pass
    else:
        if "deletions" not in diffresult[x]:
            diffres.append(line + '0 deletions')
        else:
            if "insertions" not in diffresult[x]:
                index = line.find('changed,')
                diffres.append(line[:index+8] + '0 insertions' + line[index+7:])
            else:
                diffres.append(diffresult[x])

In [10]:
kl = []
res = []
for x in range(0, len(diffres)-1):
    kl = diffres[x].split(',')
    #del kl[len(kl)-1]
    res.append(kl)
#for it in res:
#    print(it)

In [11]:
temp1hist = []
for b in range(0,len(res)-1):
    temp2 = []
    for c in range(0,len(res[b])-3):
        temp2.append(res[b][c])
    temp1hist.append(temp2)
#temp1hist

In [12]:
with open('diffhistogram.csv', 'w') as csvfile:
    header = ['commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for cmp in range(0,len(temp1hist)-1):
        x = 2
        y = 4
        if len(temp1hist[cmp])<=4:
            writers.writerow(temp1hist[cmp])
        else:
            s = (len(temp1hist[cmp])-2)/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,2):
                    v.append(temp1hist[cmp][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',temp1hist[cmp][col2]) == True:
                        v.append(re.findall(r'\d+',temp1hist[cmp][col2]))
                    else:
                        v.append(temp1hist[cmp][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

In [13]:
datahist = pd.read_csv("diffhistogram.csv")
sc = datahist[header]
sc.head()

,commitID_1,commitID_2,filename,#line_changed
0,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,14d4b8007cddcbd6197404f7ad435b9b514b2859,.../src/main/java/org/apache/openjpa/kernel/Br...,10
1,778e8e9449210bda0f3fad71081ae3dc954e8a64,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,pom.xml,6
2,2898b4e342591dc98848663caa74a9bb7fee35b4,778e8e9449210bda0f3fad71081ae3dc954e8a64,.../openjpa/jdbc/sql/SQLServerDictionary.java,23
3,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,2898b4e342591dc98848663caa74a9bb7fee35b4,.../org/apache/openjpa/persistence/criteria/Cr...,2
4,ed28ddca1a1ff1026cc068a0a85540eb756f655e,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,.../org/apache/openjpa/jdbc/meta/ClassMapping....,2


In [14]:
dfhist = pd.read_csv('diffhistogram.csv')
dfhist = (pd.to_numeric(dfhist['#line_changed'], errors='coerce')
       .groupby(dfhist['filename'])
       .sum()
       .to_frame()
       .add_prefix('SUM ')
       .reset_index())

col = ['filename','SUM #line_changed']
dfhist = dfhist.dropna(subset=['SUM #line_changed'])
dfhist = dfhist.sort_values('SUM #line_changed', ascending=False)

dfhist[col].to_csv('diffhistogramfilechanged.csv')
out = pd.read_csv('diffhistogramfilechanged.csv')
saved = out[col]
saved.head()

,filename,SUM #line_changed
0,openjpaproject/src/doc/manual/manual.xml,67857.0
1,.../java/openbook/tools/parser/JavaParser.java,20088.0
2,.../src/doc/manual/ref_guide_dbsetup.xml,12746.0
3,Bin,9705.0
4,.../src/doc/manual/jpa_overview_query.xml,9547.0


In [15]:
trace1 = go.Bar(
    x=dfhist['filename'][:10],
    y=dfhist['SUM #line_changed'][:10],
    name='--diff-algorithm=histogram'
)

trace2 = go.Scatter(
    x=dfhist['filename'][:10],
    y=dfhist['SUM #line_changed'][:10],
    name='--diff-algorithm=histogram')

dt = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)

# Applying git diff -w --diff-algorithm=myers --stat commitID_1 commitID2 

In [16]:
diffmyers = []
for i in range(0, len(com_ID)-3):
    sys.stdout.write('\rApplying diff between commits = %i' % (i+1) + ' out of %i' % (len(com_ID)-3))
    sys.stdout.flush()
    diff_cmd = "git diff -w --diff-algorithm=myers --stat " + com_ID[i+1] + " " + com_ID[i]
    tmp = (str(execute_shell_command(com_ID[i+1], com_ID[i], diff_cmd, repository)).replace("(",'').replace("+",'').
           replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').replace(")",'').
           replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
           replace("...=> ",'...=>').replace(" Code ","Code"))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                    replace(",file",'files').replace(",insertions",'insertions').
                    replace(",insertion",'insertions').replace(",deletions,",'deletions').
                    replace(",deletion,",'deletions').replace(" ",''))
    diffmyers.append(str(tmp2))
    
diffmyersresult = []
for item in diffmyers:
    if item == ', ':
        del item
    else:
        diffmyersresult.append(item)

#diffmyersresult

Applying diff between commits = 4860 out of 4860

In [17]:
diffmyersres = []
for x in range(0,len(diffmyersresult)-1):
    line = diffmyersresult[x]
    if "changed" not in diffmyersresult[x]:
        pass
    else:
        if "deletions" not in diffmyersresult[x]:
            diffmyersres.append(line + '0 deletions')
        else:
            if "insertions" not in diffmyersresult[x]:
                index = line.find('changed,')
                diffmyersres.append(line[:index+8] + '0 insertions' + line[index+7:])
            else:
                diffmyersres.append(diffmyersresult[x])

In [18]:
klmyers = []
resmyers = []
for x in range(0, len(diffmyersres)-1):
    kl = diffmyersres[x].split(',')
    resmyers.append(kl)
#for it in resmyers:
#    print(it)

In [19]:
temp1myers = []
for b in range(0,len(resmyers)-1):
    temp2myers = []
    for c in range(0,len(resmyers[b])-3):
        temp2myers.append(resmyers[b][c])
    temp1myers.append(temp2myers)
#temp1myers

In [20]:
with open('diffmyers.csv', 'w') as csvfile:
    header = ['commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for cmp in range(0,len(temp1myers)-1):
        x = 2
        y = 4
        if len(temp1myers[cmp])<=4:
            writers.writerow(temp1myers[cmp])
        else:
            s = (len(temp1myers[cmp])-2)/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,2):
                    v.append(temp1myers[cmp][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',temp1myers[cmp][col2]) == True:
                        v.append(re.findall(r'\d+',temp1myers[cmp][col2]))
                    else:
                        v.append(temp1myers[cmp][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

In [21]:
datamyers = pd.read_csv("diffmyers.csv")
sc = datamyers[header]
sc.head()

,commitID_1,commitID_2,filename,#line_changed
0,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,14d4b8007cddcbd6197404f7ad435b9b514b2859,.../src/main/java/org/apache/openjpa/kernel/Br...,10
1,778e8e9449210bda0f3fad71081ae3dc954e8a64,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,pom.xml,6
2,2898b4e342591dc98848663caa74a9bb7fee35b4,778e8e9449210bda0f3fad71081ae3dc954e8a64,.../openjpa/jdbc/sql/SQLServerDictionary.java,23
3,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,2898b4e342591dc98848663caa74a9bb7fee35b4,.../org/apache/openjpa/persistence/criteria/Cr...,2
4,ed28ddca1a1ff1026cc068a0a85540eb756f655e,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,.../org/apache/openjpa/jdbc/meta/ClassMapping....,2


In [22]:
dfmyers = pd.read_csv('diffmyers.csv')
dfmyers = (pd.to_numeric(dfmyers['#line_changed'], errors='coerce')
       .groupby(dfmyers['filename'])
       .sum()
       .to_frame()
       .add_prefix('SUM ')
       .reset_index())

col = ['filename','SUM #line_changed']
dfmyers = dfmyers.dropna(subset=['SUM #line_changed'])
dfmyers = dfmyers.sort_values('SUM #line_changed', ascending=False)

dfmyers[col].to_csv('diffmyersfilechanged.csv')
out = pd.read_csv('diffmyersfilechanged.csv')
saved = out[col]
saved.head()

,filename,SUM #line_changed
0,openjpaproject/src/doc/manual/manual.xml,67869.0
1,.../java/openbook/tools/parser/JavaParser.java,20088.0
2,.../src/doc/manual/ref_guide_dbsetup.xml,11552.0
3,Bin,9705.0
4,.../java/org/apache/openjpa/kernel/BrokerImpl....,9522.0


In [23]:
trace3 = go.Bar(
    x=dfmyers['filename'][:10],
    y=dfmyers['SUM #line_changed'][:10],
    name='--diff-algorithm=myers'
)

trace4 = go.Scatter(
    x=dfmyers['filename'][:10],
    y=dfmyers['SUM #line_changed'][:10],
    name='--diff-algorithm=myers')

dt = [trace3, trace4]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)

# Applying git diff -w --diff-algorithm=patience --stat commitID_1 commitID2

In [24]:
diffpatience = []
for i in range(0, len(com_ID)-3):
    sys.stdout.write('\rApplying diff between commits = %i' % (i+1) + ' out of %i' % (len(com_ID)-3))
    sys.stdout.flush()
    diff_cmd = "git diff -w --diff-algorithm=patience --stat " + com_ID[i+1] + " " + com_ID[i]
    tmp = (str(execute_shell_command(com_ID[i+1], com_ID[i], diff_cmd, repository)).replace("(",'').replace("+",'').
           replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').replace(")",'').
           replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
           replace("...=> ",'...=>').replace(" Code ","Code"))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                    replace(",file",'files').replace(",insertions",'insertions').
                    replace(",insertion",'insertions').replace(",deletions,",'deletions').
                    replace(",deletion,",'deletions').replace(" ",''))
    diffpatience.append(str(tmp2))
    
diffpatienceresult = []
for item in diffpatience:
    if item == ', ':
        del item
    else:
        diffpatienceresult.append(item)

Applying diff between commits = 4860 out of 4860

In [25]:
diffpatiencesres = []
for x in range(0,len(diffpatienceresult)-1):
    line = diffpatienceresult[x]
    if "changed" not in diffpatienceresult[x]:
        pass
    else:
        if "deletions" not in diffpatienceresult[x]:
            diffpatiencesres.append(line + '0 deletions')
        else:
            if "insertions" not in diffpatienceresult[x]:
                index = line.find('changed,')
                diffpatiencesres.append(line[:index+8] + '0 insertions' + line[index+7:])
            else:
                diffpatiencesres.append(diffpatienceresult[x])

In [26]:
klpatience = []
respatience = []
for x in range(0, len(diffpatiencesres)-1):
    klpatience = diffpatiencesres[x].split(',')
    respatience.append(klpatience)

In [27]:
temp1patience = []
for b in range(0,len(respatience)-1):
    temp2patience = []
    for c in range(0,len(respatience[b])-3):
        temp2patience.append(respatience[b][c])
    temp1patience.append(temp2patience)

In [28]:
with open('diffpatience.csv', 'w') as csvfile:
    header = ['commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for cmp in range(0,len(temp1patience)-1):
        x = 2
        y = 4
        if len(temp1patience[cmp])<=4:
            writers.writerow(temp1patience[cmp])
        else:
            s = (len(temp1patience[cmp])-2)/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,2):
                    v.append(temp1patience[cmp][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',temp1patience[cmp][col2]) == True:
                        v.append(re.findall(r'\d+',temp1patience[cmp][col2]))
                    else:
                        v.append(temp1patience[cmp][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

In [29]:
datapatience = pd.read_csv("diffpatience.csv")
sc = datapatience[header]
sc.head()

,commitID_1,commitID_2,filename,#line_changed
0,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,14d4b8007cddcbd6197404f7ad435b9b514b2859,.../src/main/java/org/apache/openjpa/kernel/Br...,10
1,778e8e9449210bda0f3fad71081ae3dc954e8a64,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,pom.xml,6
2,2898b4e342591dc98848663caa74a9bb7fee35b4,778e8e9449210bda0f3fad71081ae3dc954e8a64,.../openjpa/jdbc/sql/SQLServerDictionary.java,23
3,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,2898b4e342591dc98848663caa74a9bb7fee35b4,.../org/apache/openjpa/persistence/criteria/Cr...,2
4,ed28ddca1a1ff1026cc068a0a85540eb756f655e,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,.../org/apache/openjpa/jdbc/meta/ClassMapping....,2


In [30]:
dfpatience = pd.read_csv('diffpatience.csv')
dfpatience = (pd.to_numeric(dfpatience['#line_changed'], errors='coerce')
       .groupby(dfpatience['filename'])
       .sum()
       .to_frame()
       .add_prefix('SUM ')
       .reset_index())

col = ['filename','SUM #line_changed']
dfpatience = dfpatience.dropna(subset=['SUM #line_changed'])
dfpatience = dfpatience.sort_values('SUM #line_changed', ascending=False)

dfpatience[col].to_csv('diffpatiencefilechanged.csv')
out = pd.read_csv('diffpatiencefilechanged.csv')
saved = out[col]
saved.head()

,filename,SUM #line_changed
0,openjpaproject/src/doc/manual/manual.xml,67857.0
1,.../java/openbook/tools/parser/JavaParser.java,20088.0
2,.../src/doc/manual/ref_guide_dbsetup.xml,12120.0
3,Bin,9705.0
4,.../java/org/apache/openjpa/kernel/BrokerImpl....,9524.0


In [31]:
trace5 = go.Bar(
    x=dfpatience['filename'][:10],
    y=dfpatience['SUM #line_changed'][:10],
    name='--diff-algorithm=patience'
)

trace6 = go.Scatter(
    x=dfpatience['filename'][:10],
    y=dfpatience['SUM #line_changed'][:10],
    name='--diff-algorithm=patience')

dt = [trace5, trace6]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)

# Applying git diff -w --diff-algorithm=minimal --stat commitID_1 commitID2

In [32]:
diffminimal = []
for i in range(0, len(com_ID)-3):
    sys.stdout.write('\rApplying diff between commits = %i' % (i+1) + ' out of %i' % (len(com_ID)-3))
    sys.stdout.flush()
    diff_cmd = "git diff -w --diff-algorithm=minimal --stat " + com_ID[i+1] + " " + com_ID[i]
    tmp = (str(execute_shell_command(com_ID[i+1], com_ID[i], diff_cmd, repository)).replace("(",'').replace("+",'').
           replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').replace(")",'').
           replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
           replace("...=> ",'...=>').replace(" Code ","Code"))
    tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
    tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                    replace(",file",'files').replace(",insertions",'insertions').
                    replace(",insertion",'insertions').replace(",deletions,",'deletions').
                    replace(",deletion,",'deletions').replace(" ",''))
    diffminimal.append(str(tmp2))
    
diffminresult = []
for item in diffminimal:
    if item == ', ':
        del item
    else:
        diffminresult.append(item)

Applying diff between commits = 4860 out of 4860

In [33]:
diffminsres = []
for x in range(0,len(diffminresult)-1):
    line = diffminresult[x]
    if "changed" not in diffminresult[x]:
        pass
    else:
        if "deletions" not in diffminresult[x]:
            diffminsres.append(line + '0 deletions')
        else:
            if "insertions" not in diffminresult[x]:
                index = line.find('changed,')
                diffminsres.append(line[:index+8] + '0 insertions' + line[index+7:])
            else:
                diffminsres.append(diffminresult[x])

In [34]:
klminimal = []
resminimal = []
for x in range(0, len(diffminsres)-1):
    klminimal = diffminsres[x].split(',')
    resminimal.append(klminimal)

In [35]:
temp1min = []
for b in range(0,len(resminimal)-1):
    temp2min = []
    for c in range(0,len(resminimal[b])-3):
        temp2min.append(resminimal[b][c])
    temp1min.append(temp2min)

In [36]:
with open('diffminimal.csv', 'w') as csvfile:
    header = ['commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for cmp in range(0,len(temp1min)-1):
        x = 2
        y = 4
        if len(temp1min[cmp])<=4:
            writers.writerow(temp1min[cmp])
        else:
            s = (len(temp1min[cmp])-2)/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,2):
                    v.append(temp1min[cmp][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',temp1min[cmp][col2]) == True:
                        v.append(re.findall(r'\d+',temp1min[cmp][col2]))
                    else:
                        v.append(temp1min[cmp][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

In [37]:
dataminimal = pd.read_csv("diffminimal.csv")
sc = dataminimal[header]
sc.head()

,commitID_1,commitID_2,filename,#line_changed
0,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,14d4b8007cddcbd6197404f7ad435b9b514b2859,.../src/main/java/org/apache/openjpa/kernel/Br...,10
1,778e8e9449210bda0f3fad71081ae3dc954e8a64,b4c13fc0ed80fbfe59df1ce960454c21a0504b56,pom.xml,6
2,2898b4e342591dc98848663caa74a9bb7fee35b4,778e8e9449210bda0f3fad71081ae3dc954e8a64,.../openjpa/jdbc/sql/SQLServerDictionary.java,23
3,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,2898b4e342591dc98848663caa74a9bb7fee35b4,.../org/apache/openjpa/persistence/criteria/Cr...,2
4,ed28ddca1a1ff1026cc068a0a85540eb756f655e,5c94a7ff67ad38f6fdcaa7a950eeb9622ff960aa,.../org/apache/openjpa/jdbc/meta/ClassMapping....,2


In [38]:
dfminimal = pd.read_csv('diffminimal.csv')
dfminimal = (pd.to_numeric(dfminimal['#line_changed'], errors='coerce')
       .groupby(dfminimal['filename'])
       .sum()
       .to_frame()
       .add_prefix('SUM ')
       .reset_index())

col = ['filename','SUM #line_changed']
dfminimal = dfminimal.dropna(subset=['SUM #line_changed'])
dfminimal = dfminimal.sort_values('SUM #line_changed', ascending=False)

dfminimal[col].to_csv('diffminimalfilechanged.csv')
out = pd.read_csv('diffminimalfilechanged.csv')
saved = out[col]
saved.head()

,filename,SUM #line_changed
0,openjpaproject/src/doc/manual/manual.xml,67857.0
1,.../java/openbook/tools/parser/JavaParser.java,20088.0
2,.../src/doc/manual/ref_guide_dbsetup.xml,11378.0
3,Bin,9705.0
4,.../java/org/apache/openjpa/kernel/BrokerImpl....,9522.0


In [39]:
trace7 = go.Bar(
    x=dfminimal['filename'][:10],
    y=dfminimal['SUM #line_changed'][:10],
    name='--diff-algorithm=minimal'
)

trace8 = go.Scatter(
    x=dfminimal['filename'][:10],
    y=dfminimal['SUM #line_changed'][:10],
    name='--diff-algorithm=minimal')

dt = [trace7, trace8]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)

In [40]:
dtcombine = [trace3, trace1, trace5, trace7]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dtcombine, layout=layout)
iplot(fig, show_link=False)